## Key point based data augmentation for Distracted Driver Detection

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [2]:
from keras.optimizers import RMSprop

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('keys.txt', delimiter="(",  header=None)

In [5]:
for i in range(5):
    data[i] = data[i].astype(str).str[:-1]
data.head()

,0,1,2,3,4
0,img_56.jpg,"192,178","461,126","443,219","587,321"
1,img_63.jpg,"333,135","636,56","581,171","637,383"
2,img_118.jpg,"242,134","539,161","540,240","633,398"
3,img_120.jpg,"163,132","479,110","440,213","592,304"
4,img_138.jpg,"192,185","514,105","490,213","588,407"


In [6]:
data.columns = ["image","Head","left_hand","steering_wheel","right_hand"]

In [7]:
data.head()

,image,Head,left_hand,steering_wheel,right_hand
0,img_56.jpg,"192,178","461,126","443,219","587,321"
1,img_63.jpg,"333,135","636,56","581,171","637,383"
2,img_118.jpg,"242,134","539,161","540,240","633,398"
3,img_120.jpg,"163,132","479,110","440,213","592,304"
4,img_138.jpg,"192,185","514,105","490,213","588,407"


In [8]:
data["Head_x"], data["Head_y"] = data['Head'].str.split(',', 1).str
data["left_hand_x"], data["left_hand_y"] = data['left_hand'].str.split(',', 1).str
data["steering_wheel_x"], data["steering_wheel_y"] = data['steering_wheel'].str.split(',', 1).str
data["right_hand_x"], data["right_hand_y"] = data['right_hand'].str.split(',', 1).str
data.head()

,image,Head,left_hand,steering_wheel,right_hand,Head_x,Head_y,left_hand_x,left_hand_y,steering_wheel_x,steering_wheel_y,right_hand_x,right_hand_y
0,img_56.jpg,"192,178","461,126","443,219","587,321",192,178,461,126,443,219,587,321
1,img_63.jpg,"333,135","636,56","581,171","637,383",333,135,636,56,581,171,637,383
2,img_118.jpg,"242,134","539,161","540,240","633,398",242,134,539,161,540,240,633,398
3,img_120.jpg,"163,132","479,110","440,213","592,304",163,132,479,110,440,213,592,304
4,img_138.jpg,"192,185","514,105","490,213","588,407",192,185,514,105,490,213,588,407


In [9]:
data.drop("Head",axis=1,inplace=True)
data.drop("left_hand",axis=1,inplace=True)
data.drop("steering_wheel",axis=1,inplace=True)
data.drop("right_hand",axis=1,inplace=True)
data.head()

,image,Head_x,Head_y,left_hand_x,left_hand_y,steering_wheel_x,steering_wheel_y,right_hand_x,right_hand_y
0,img_56.jpg,192,178,461,126,443,219,587,321
1,img_63.jpg,333,135,636,56,581,171,637,383
2,img_118.jpg,242,134,539,161,540,240,633,398
3,img_120.jpg,163,132,479,110,440,213,592,304
4,img_138.jpg,192,185,514,105,490,213,588,407


In [123]:
def key_model():
    model=Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(3,150,150)))
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))

    # the model so far outputs 3D feature maps (height, width, features)
    model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64)) # 64 neurons
    model.add(Activation('relu'))
    model.add(Dense(64)) # 64 neurons
    model.add(Activation('relu'))
    model.add(Dense(32)) # 64 neurons
    model.add(Activation('relu'))

    # output layer: get 8 coordinate values
    model.add(Dense(8))
    model.compile(loss='mean_squared_error', optimizer=RMSprop())
    return model

In [77]:
data.values

array([['img_56.jpg', '192', '178', ..., '219', '587', '321'],
       ['img_63.jpg', '333', '135', ..., '171', '637', '383'],
       ['img_118.jpg', '242', '134', ..., '240', '633', '398'],
       ..., 
       ['img_20722.jpg', '206', '148', ..., '217', '592', '336'],
       ['img_20750.jpg', '216', '108', ..., '164', '625', '291'],
       ['img_20763.jpg', '288', '92', ..., '170', '628', '339']], dtype=object)

In [10]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score, KFold
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
seed = 42

In [11]:
X = data["image"].values

In [12]:
data.drop("image",axis=1, inplace=True)

In [13]:
data.head()

,Head_x,Head_y,left_hand_x,left_hand_y,steering_wheel_x,steering_wheel_y,right_hand_x,right_hand_y
0,192,178,461,126,443,219,587,321
1,333,135,636,56,581,171,637,383
2,242,134,539,161,540,240,633,398
3,163,132,479,110,440,213,592,304
4,192,185,514,105,490,213,588,407


In [14]:
Y = data.values

In [15]:
Y

array([['192', '178', '461', ..., '219', '587', '321'],
       ['333', '135', '636', ..., '171', '637', '383'],
       ['242', '134', '539', ..., '240', '633', '398'],
       ..., 
       ['206', '148', '464', ..., '217', '592', '336'],
       ['216', '108', '535', ..., '164', '625', '291'],
       ['288', '92', '576', ..., '170', '628', '339']], dtype=object)

In [124]:
estimator = KerasRegressor(build_fn=key_model, nb_epoch=100, batch_size=100, verbose=False)

In [16]:
kfold = KFold(n_splits=10, random_state=seed)

In [ ]:
import numpy as np
from PIL import Image
x = np.array([np.array(Image.open(fname).resize((150,150),Image.ANTIALIAS)) for fname in "key_data/"+X])

In [134]:
x = x.transpose(0,3,1,2)
x.shape

(499, 3, 150, 150)

In [135]:
x

array([[[[ 40,  38,  38, ..., 249, 145,  13],
         [ 39,  39,  38, ..., 236, 139,  14],
         [ 38,  39,  39, ..., 216, 131,  13],
         ..., 
         [  5,   5,   5, ...,  16,  14,  10],
         [  5,   5,   5, ...,  15,  18,  13],
         [  5,   5,   5, ...,  16,  19,  17]],

        [[ 48,  46,  47, ..., 255, 158,  30],
         [ 47,  47,  47, ..., 255, 159,  31],
         [ 46,  47,  47, ..., 255, 160,  30],
         ..., 
         [  6,   6,   6, ...,  16,  15,  11],
         [  6,   6,   6, ...,  13,  18,  14],
         [  6,   6,   6, ...,  13,  19,  17]],

        [[ 33,  31,  31, ..., 255, 160,  24],
         [ 33,  33,  32, ..., 255, 162,  27],
         [ 34,  35,  34, ..., 255, 164,  29],
         ..., 
         [  1,   1,   1, ...,  13,   9,   4],
         [  1,   1,   1, ...,  18,  16,   8],
         [  1,   1,   1, ...,  25,  23,  16]]],


       [[[ 27,  27,  35, ..., 165, 184, 218],
         [ 28,  22,  14, ..., 171, 184, 203],
         [ 30,  44, 102, ..

In [106]:
Y.shape

(499, 8)

In [136]:
results = cross_val_score(estimator, x, Y, cv=kfold)

ResourceExhaustedError: OOM when allocating tensor with shape[100,146,146,64]
	 [[Node: conv2d_26/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](conv2d_26/transpose, conv2d_26/kernel/read)]]

Caused by op 'conv2d_26/convolution', defined at:
  File "E:\Software\Conda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "E:\Software\Conda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "E:\Software\Conda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "E:\Software\Conda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "E:\Software\Conda\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loop.start()
  File "E:\Software\Conda\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start

In [137]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14012734960775855169]